In [7]:
import numpy as np 
import math 
import rospy 
import rospy
from tf import transformations
from std_msgs.msg import Header, ColorRGBA
from visualization_msgs.msg import Marker, MarkerArray
import sys
sys.path.append("..")
from kinematics.utils.util_ik import make_ik_input
from kinematics.class_structure import CHAIN
from kinematics.class_robot import RobotClass
from kinematics.utils.util_ik import add_joints, find_route
from kinematics.utils.util_rviz import publish_viz_robot, publish_viz_markers
from kinematics.utils.util_structure import update_q_chain, get_p_chain, get_R_chain, get_rpy_from_R_mat, get_mesh_chain, get_scale, get_link_color, get_viz_ingredients
print("Done.")

Done.


In [8]:
class RvizClass:
    def __init__(self, file_name = "../urdf/ur5e/ur5e_onrobot.urdf", base_offset=[0,0,0]):
        rospy.init_node("Run_Robot")
        self.pub_robot      = rospy.Publisher('viz_robot', MarkerArray, queue_size=10)
        self.pub_sphere     = rospy.Publisher('viz_sphere', MarkerArray, queue_size=10)
        self.chain          = CHAIN(file_name=file_name, base_offset=base_offset, verbose=False)
        self.chain.add_joi_to_robot()
        self.chain.add_link_to_robot()
        self.ctrl_joint_num = 6

    def check_fk(self, q_list):
        update_q_chain(self.chain.joint, q_list, self.ctrl_joint_num)
        self.chain.fk_chain(1)
        p_list     = get_p_chain(self.chain.joint)
        return p_list 

    def publish_robot(self, q_list):
        update_q_chain(self.chain.joint, q_list, self.ctrl_joint_num)
        self.chain.fk_chain(1)
        p_list     = get_p_chain(self.chain.joint)
        R_list     = get_R_chain(self.chain.joint)
        rpy_list   = get_rpy_from_R_mat(R_list)
        mesh_list  = get_mesh_chain(self.chain.link)
        scale_list = get_scale(self.chain.link)
        color_list = get_link_color(self.chain.link)
        viz_links  =  get_viz_ingredients(p_list, rpy_list, mesh_list, scale_list, color_list)
        viz_trg_robot = publish_viz_robot(viz_links)
        self.pub_robot.publish(viz_trg_robot)

    def publish_markers(self, obj):
        viz_obj = publish_viz_markers(obj)
        self.pub_sphere.publish(viz_obj)

def make_markers(name, type, pos, rot, size, color): 
    return {"name":name, "type":type, "info":pos+rot+size, "color":color}


In [9]:
file_name = "../urdf/ur5e/ur5e_onrobot.urdf"
base_offset = [0.18,0,0.79]
robot     = RobotClass(file_name=file_name, base_offset=base_offset)
rviz = RvizClass(file_name=file_name, base_offset=base_offset)

q = robot.solve_ik(target_name = ['wrist_3_joint', 'gripper_tcp_joint'],
            target_position  = [[0.6,0.,1.5],[0,0,0]],
            target_rotation  = [[0,0,0],[0, -math.pi/6-math.pi, 0]],
            solve_position   = [1,0],
            solve_rotation   = [0,1],
            weight_position  = 1,
            weight_rotation  = 1,
            joi_ctrl_num= 6)
# q         = robot.solve_ik(target_name = ['gripper_tcp_joint'],
#             target_position  = [[0.4+0.18,0.,1.4]],
#             target_rotation  = [[0, -math.pi/4-math.pi, 0]],
#             solve_position   = [1],
#             solve_rotation   = [1],
#             weight_position  = 1,
#             weight_rotation  = 1,
#             joi_ctrl_num= 6)

q_list = q  #np.array([0, 0, -math.pi, math.pi/2, -math.pi/2, -math.pi/2, 0])
rviz.publish_robot(q_list=q_list)
print("Control Joint: {}/Shape: {}".format(q_list, q_list.shape))
print("Position of {}: {}".format(robot.chain.joint[9].name, [round(robot.chain.joint[9].p.T[0][0],3),
                                                              round(robot.chain.joint[9].p.T[0][1],3), 
                                                              round(robot.chain.joint[9].p.T[0][2],3)]))
print("Position of {}: {}".format(robot.chain.joint[6].name, [round(robot.chain.joint[6].p.T[0][0],3),
                                                              round(robot.chain.joint[6].p.T[0][1],3), 
                                                              round(robot.chain.joint[6].p.T[0][2],3)]))


Control Joint: [ 0.         -0.32472405 -0.55274419 -0.69599508  2.31908853  1.40982282
 -1.85450891]/Shape: (7,)
Position of gripper_tcp_joint: [0.743, 0.0, 1.253]
Position of wrist_3_joint: [0.6, 0.0, 1.5]


In [10]:
tcp_position = rviz.chain.joint[9].p.reshape(3,).tolist()
obs_info  = make_markers(name="tcp_marker", 
                         type="cube",  
                         pos=tcp_position, 
                         rot=[0,0,0], 
                         size=[0.025, 0.025, 0.025], 
                         color=[0.5,0,0,1])

obs_info_lst = [obs_info]        

START = True

""" VISUALIZE """
if START:
    rendering=0
    rate = rospy.Rate(50)
    while not rospy.is_shutdown():
        if rendering == 500:
            break 
        rviz.publish_robot(q_list=q_list)
        rviz.publish_markers(obs_info_lst)
        rendering +=1
        rate.sleep()


In [12]:
""" VISUALIZE OBSTACLES"""
obs_info  = make_markers(name="marker1", 
                         type="cube",  
                         pos=[1.45, 0, 0.7], 
                         rot=[0,0,0], 
                         size=[0.05, 0.05, 0.05], 
                         color=[0.5,0,0,1])
                         
obs_info2 = make_markers(name="marker2", 
                         type="sphere", 
                         pos=[0.25, 0.6, 0.7], 
                         rot=[0,0,0], 
                         size=[0.05, 0.05, 0.05], 
                         color=[0,0.5,0,1])

obs_info3 = make_markers(name="marker3", 
                         type="sphere", 
                         pos=[-0.45, 0.1, 0.7], 
                         rot=[0,0,0], 
                         size=[0.05, 0.05, 0.05], 
                         color=[0.,0,0.5,1])

obs_info_lst2 = [obs_info, obs_info2, obs_info3] 

OBS_VIZ= False 
if OBS_VIZ:
    rendering=0
    rate = rospy.Rate(10)
    while not rospy.is_shutdown():
        if rendering == 100:
            break 
        rviz.publish_markers(obs_info_lst2)
        rendering +=1
        rate.sleep()

print("Done.")

Done.
